## Aim

I aim to find the bug in 2014-2018 session codes. I found it and have updated the script. 

In [1]:
import pandas as pd
import numpy as np
import time 
import math
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select

In [2]:
driver = webdriver.Firefox()
wait = WebDriverWait(driver, 10)

In [3]:
driver.get('https://convention2.allacademic.com/one/ica/ica14/')

In [4]:
def click_browse_by_session_type():
    '''click on "browse by session type"
    '''
    browse_by_session_type = driver.find_elements(
        By.CSS_SELECTOR, "li.ui-li-has-icon.ui-last-child > a"
    )[3]
    browse_by_session_type.click()

def click_paper_session():
    '''click "paper session" button
    '''
    paper_session = driver.find_element(
        By.XPATH, '//li[@class="ui-li-has-count"][3] //a[@class="ui-btn"]'
    )
    paper_session.click()

def get_sessions():
    '''These are session links
    '''
    sessions = driver.find_elements(
        By.CSS_SELECTOR, 'a.ul-li-has-alt-left.ui-btn'
    )
    return sessions

In [5]:
click_browse_by_session_type()
click_paper_session()
sessions = get_sessions()

In [6]:
len(sessions)

228

In [7]:
s = sessions[0]

In [8]:
s_link = s.get_attribute('href')

In [9]:
# open a new window
driver.execute_script("window.open('');")
# switch to the new window
driver.switch_to.window(driver.window_handles[1])
# open the session
driver.get(s_link)

In [10]:
session_title_e = driver.find_element(
    By.CSS_SELECTOR, 'h3'
)
session_title = session_title_e.text

# sub unit, cosponsor, chair, the presentations
h4s = driver.find_elements(
    By.CSS_SELECTOR, 'h4'
)
h4s_texts = [i.text for i in h4s]
sub_unit_e_idx = h4s_texts.index('Sub Unit')

In [11]:
session_title

'Meda Coverage of Health Issues'

In [13]:
h4s_texts

['Sub Unit', 'Chair', 'Individual Presentations']

In [14]:
sub_unit_e_idx

0

In [15]:
try:
    sub_unit_e = driver.find_elements(
        By.CSS_SELECTOR, 'ul.ui-listview.ui-listview-inset.ui-corner-all.ui-shadow'
    )[4]
    sub_unit = sub_unit_e.text
except:
    sub_unit = None

In [16]:
sub_unit

'Health Communication'

In [17]:
if 'Cosponsor' in h4s_texts:
    chair_e_idx = 6
else:
    chair_e_idx = 5
# chair_e_idx = h4s_texts.index('Chair')
chair_graybox = driver.find_elements(
    By.CSS_SELECTOR, 'ul.ui-listview.ui-listview-inset.ui-corner-all.ui-shadow'
)[chair_e_idx]
chair_es = chair_graybox.find_elements(
    By.CSS_SELECTOR, 'li'
)

In [30]:
chair_es

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="a8724699-22cb-457f-b006-d03c52012902", element="d6da1928-786f-46c2-9844-2205f0c4e621")>]

In [31]:
len(chair_es) == 1

True

In [34]:
if chair_es:
    if len(chair_es) == 1:
        chair_info = chair_es[0].text
        chair_name = chair_info.split(', ')[0]
        chair_aff = chair_info.split(', ')[1]
    # this is to solve the issue of when there are multiple chairs. For example,
    # year 2018, session 'Research Escalator - Part 1'
    else:
        chair_name = ''
        chair_aff = ''
        for chair_e in chair_es:
            chair_info = chair_e.text
            chair_name_i = chair_info.split(', ')[0]
            chair_aff_i = chair_info.split(', ')[1]
            chair_name += chair_name_i
            chair_aff += chair_aff_i
            if chair_e != chair_es[-1]:
                chair_name += '; '
                chair_aff += '; '